In [1]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import gzip
import Preprocessing as pproc 

directory = os.getcwd()
data_path = os.path.join(directory, 'Data')

# Definining the path for the files
RB_path = os.path.join(data_path, 'RateBeer')
BA_path = os.path.join(data_path, 'BeerAdvocate')
MB_path = os.path.join(data_path, 'matched_beer_data')

In [2]:
#Transformation of the .txt files
pproc.text_to_csv(RB_path, 'ratings')
pproc.text_to_csv(RB_path, 'reviews')

pproc.text_to_csv(BA_path, 'ratings')
pproc.text_to_csv(BA_path, 'reviews')

pproc.text_to_csv(MB_path, 'ratings_ba')
pproc.text_to_csv(MB_path, 'ratings_with_text_ba')
pproc.text_to_csv(MB_path, 'ratings_rb')
pproc.text_to_csv(MB_path, 'ratings_with_text_rb')

In [38]:
#MATCHING BEERS:
MB_beers = pd.read_csv(MB_path + '/beers.csv', header=1)
MB_breweries = pd.read_csv(MB_path + '/breweries.csv', header=1)
MB_users = pd.read_csv(MB_path + '/users.csv', header=1)
MB_users_approx = pd.read_csv(MB_path + '/users_approx.csv', header=1)
MB_ratings = pd.read_csv(MB_path + '/ratings.csv', header=1)


In [4]:
#RATEBEERS WEB:
RB_beers = pd.read_csv(RB_path + '/beers.csv')
RB_breweries = pd.read_csv(RB_path + '/breweries.csv')
RB_users = pd.read_csv(RB_path + '/users.csv')

In [15]:
#BEERADVOCATES WEB:
BA_beers = pd.read_csv(BA_path + '/beers.csv')
BA_breweries = pd.read_csv(BA_path + '/breweries.csv')
BA_users = pd.read_csv(BA_path + '/users.csv')
BA_ratings = pd.read_csv(BA_path + '/ratings.tsv', sep='\t')
BA_reviews = pd.read_csv(BA_path + '/reviews.tsv', sep='\t')

In [59]:
MB_ratings['date'] =pd.to_datetime(MB_ratings.date, unit='s')
MB_ratings['year'] = MB_ratings['date'].dt.year
columns_to_keep = ['user_id', 'rating','year']
filter_MB_ratings = MB_ratings.filter(columns_to_keep)


In [60]:
filter_MB_ratings.head(2)

,user_id,rating,year
0,erzengel.248045,4.8,1970
1,visionthing.639993,3.0,1970


So with the previous datasets we have the geographical location and the brewery name. Furthermore, it can be merged by the userID that coincides in both columns.

In [73]:
geo_beers = pd.merge(filter_MB_ratings, MB_users[['user_id','location']], on='user_id', how='outer')

In [74]:
geo_beers

,user_id,rating,year,location
0,erzengel.248045,4.80,1970.0,Germany
1,visionthing.639993,3.00,1970.0,Estonia
2,tiong.608427,3.85,1970.0,Finland
3,tiong.608427,3.68,1970.0,Finland
4,tiong.608427,4.00,1970.0,Finland
...,...,...,...,...
24100,shadesofgray.371155,NaN,NaN,"United States, Pennsylvania"
24101,ypsifly.8833,NaN,NaN,"United States, Michigan"
24102,hardtarget.1712,NaN,NaN,"United States, Texas"
24103,nelsonsaigon.897449,NaN,NaN,Vietnam
